In [ ]:
# | default_exp _helpers.api_docs_helper

In [ ]:
# | export

from typing import *
import re
import types
from inspect import Signature, getmembers, isclass, isfunction, signature, getsource, getsourcefile
import textwrap

from griffe.dataclasses import Docstring
from griffe.docstrings.parsers import Parser, parse
from griffe.docstrings.dataclasses import (
    DocstringSectionAdmonition,
    DocstringSectionText,
    DocstringSectionParameters,
    DocstringSectionReturns,
    DocstringSectionRaises,
    DocstringSectionExamples,
    DocstringSectionYields,
    DocstringRaise,
    DocstringParameter
)

from nbdev.config import get_config

In [ ]:
from tempfile import TemporaryDirectory
import unittest.mock
from contextlib import contextmanager

from nbdev_mkdocs.mkdocs import prepare

In [ ]:
# | export


def _convert_union_to_optional(annotation_str: str) -> str:
    """Convert the 'Union[Type1, Type2, ..., NoneType]' to 'Optional[Type1, Type2, ...]' in the given annotation string

    Args:
        annotation_str: The type annotation string to convert.

    Returns:
        The converted type annotation string.
    """
    pattern = r"Union\[(.*)?,\s*NoneType\s*\]"
    match = re.search(pattern, annotation_str)
    if match:
        union_type = match.group(1)
        optional_type = f"Optional[{union_type}]"
        return re.sub(pattern, optional_type, annotation_str)
    else:
        return annotation_str

In [ ]:
fixtures = [
    {
        "input": "arg_1: Union[int, NoneType] = 80",
        "expected": "arg_1: Optional[int] = 80",
    },
    {
        "input": "arg_1: Union[Dict[str, str], NoneType]",
        "expected": "arg_1: Optional[Dict[str, str]]",
    },
    {
        "input": "arg_1: Union[Dict[str, str], str]",
        "expected": "arg_1: Union[Dict[str, str], str]",
    },
    {
        "input": "arg_1: str",
        "expected": "arg_1: str",
    },
    {
        "input": "arg_1: bool = False",
        "expected": "arg_1: bool = False",
    },
    {
        "input": "prefix: str = 'to_'",
        "expected": "prefix: str = 'to_'",
    },
]

for fixture in fixtures:
    actual = _convert_union_to_optional(fixture["input"])
    print(actual)
    assert actual == fixture["expected"]

arg_1: Optional[int] = 80
arg_1: Optional[Dict[str, str]]
arg_1: Union[Dict[str, str], str]
arg_1: str
arg_1: bool = False
prefix: str = 'to_'


In [ ]:
# | export


def _get_arg_list_with_signature(_signature: Signature, return_as_list: bool = False) -> Union[str, List[str]]:
    """Converts a function's signature into a string representation of its argument list.

    Args:
        _signature (signature): The signature object for the function to convert.

    Returns:
        str: A string representation of the function's argument list.
    """
    arg_list = []
    for param in _signature.parameters.values():
        arg_list.append(_convert_union_to_optional(str(param)))

    return arg_list if return_as_list else ", ".join(arg_list)

In [ ]:
def fixture_function(
    arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80
) -> str:
    pass


_signature = signature(fixture_function)

expected = (
    "arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80"
)
actual = _get_arg_list_with_signature(_signature)

print(actual)
assert actual == expected

arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80


In [ ]:
def fixture_function(
    arg_1: str, arg_2, arg_3: Union[Dict[str, str], str], arg_4: Optional[int] = 80
) -> str:
    pass


_signature = signature(fixture_function)

expected = [
    "arg_1: str",
    "arg_2",
    "arg_3: Union[Dict[str, str], str]",
    "arg_4: Optional[int] = 80",
]
actual = _get_arg_list_with_signature(_signature, True)

print(actual)
assert actual == expected

['arg_1: str', 'arg_2', 'arg_3: Union[Dict[str, str], str]', 'arg_4: Optional[int] = 80']


In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass


_signature = signature(fixture_function)

expected = "arg_1: str, arg_2"
actual = _get_arg_list_with_signature(_signature)

print(actual)
assert actual == expected

arg_1: str, arg_2


In [ ]:
# | export

def _get_return_annotation(sig: Signature, symbol_definition: str) -> str:
    if sig.return_annotation and "inspect._empty" not in str(
        sig.return_annotation
    ):
        if isinstance(sig.return_annotation, type):
            symbol_definition = symbol_definition + f" -> {sig.return_annotation.__name__}\n"
        else:
            symbol_definition = symbol_definition + f" -> {sig.return_annotation}\n"
            symbol_definition = symbol_definition.replace("typing.", "")

    else:
        symbol_definition = symbol_definition + " -> None\n"
    return symbol_definition

In [ ]:
def fixture_function() -> str:
    pass

sig = signature(fixture_function)
symbol_definition = ""
actual = _get_return_annotation(sig, symbol_definition)
expected = " -> str\n"
print(actual)

assert actual == expected

 -> str



In [ ]:
def fixture_function() -> List[str]:
    pass

sig = signature(fixture_function)
symbol_definition = ""
actual = _get_return_annotation(sig, symbol_definition)
expected = " -> List[str]\n"
print(actual)

assert actual == expected

 -> List[str]



In [ ]:
# | export


def _get_symbol_definition(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Return the definition of a given symbol.

    Args:
        symbol: A function or method object to get the definition for.

    Returns:
        A string representing the function definition
    """
    _signature = signature(symbol)
    arg_list = _get_arg_list_with_signature(_signature)
    ret_val = ""

    if isfunction(symbol):
        ret_val = f"### `{symbol.__name__}`\n\n"
        ret_val = ret_val + f"`def {symbol.__name__}({arg_list})"
        ret_val = _get_return_annotation(_signature, ret_val) + "`"
        
    return ret_val

In [ ]:
def fixture_function(arg_1: str, arg_2) -> None:
    pass


actual = _get_symbol_definition(fixture_function)
expected = """### `fixture_function`

`def fixture_function(arg_1: str, arg_2) -> None
`"""

print(actual)
assert actual == expected

### `fixture_function`

`def fixture_function(arg_1: str, arg_2) -> None
`


In [ ]:
def fixture_function(arg_1: str, arg_2) -> int:
    pass


actual = _get_symbol_definition(fixture_function)
expected = """### `fixture_function`

`def fixture_function(arg_1: str, arg_2) -> int
`"""

print(actual)
assert actual == expected

### `fixture_function`

`def fixture_function(arg_1: str, arg_2) -> int
`


In [ ]:
def fixture_function(arg_1: str, arg_2) -> "Tester":
    pass


actual = _get_symbol_definition(fixture_function)
expected = """### `fixture_function`

`def fixture_function(arg_1: str, arg_2) -> Tester
`"""

print(actual)
assert actual == expected
     

### `fixture_function`

`def fixture_function(arg_1: str, arg_2) -> Tester
`


In [ ]:
def __fixture_function__(arg_1: str, arg_2) -> "Tester":
    pass


actual = _get_symbol_definition(__fixture_function__)
expected = """### `__fixture_function__`

`def __fixture_function__(arg_1: str, arg_2) -> Tester
`"""

print(actual)
assert actual == expected
     

### `__fixture_function__`

`def __fixture_function__(arg_1: str, arg_2) -> Tester
`


In [ ]:
def fixture_function(
    arg_1: int,
    arg_2: str = "default_string",
    arg_3: Dict[str, int] = {},
    arg_4: Optional[float] = None,
    arg_5: Tuple[int, str, float] = (1, "string", 2.0),
    arg_6: List[Union[int, str]] = [1, "string"],
    arg_7: Set[int] = {1, 2, 3},
    arg_8: Union[int, str] = "string",
) -> None:
    pass


actual = _get_symbol_definition(fixture_function)
expected = """### `fixture_function`

`def fixture_function(arg_1: int, arg_2: str = 'default_string', arg_3: Dict[str, int] = {}, arg_4: Optional[float] = None, arg_5: Tuple[int, str, float] = (1, 'string', 2.0), arg_6: List[Union[int, str]] = [1, 'string'], arg_7: Set[int] = {1, 2, 3}, arg_8: Union[int, str] = 'string') -> None
`"""

print(actual)
assert actual == expected

### `fixture_function`

`def fixture_function(arg_1: int, arg_2: str = 'default_string', arg_3: Dict[str, int] = {}, arg_4: Optional[float] = None, arg_5: Tuple[int, str, float] = (1, 'string', 2.0), arg_6: List[Union[int, str]] = [1, 'string'], arg_7: Set[int] = {1, 2, 3}, arg_8: Union[int, str] = 'string') -> None
`


In [ ]:
# | export

def _format_raises_section(
    section_dict: Dict[str, Union[str, List[DocstringRaise]]]
) -> str:
    ret_val = """| Type | Description |
| --- | --- |
"""

    ret_val += "\n".join(
        [
            f"| `{v.as_dict()['annotation']}` | {v.as_dict()['description']} |" # type: ignore
            for v in section_dict["value"]
        ]
    )
    
    return ret_val

In [ ]:
def fixture_function():
    """Test funcion
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
    """
    pass

docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[1].as_dict()

actual = _format_raises_section(item_dict)
expected = """| Type | Description |
| --- | --- |
| `ValueError` | If the object has no docstring |
| `HTTPError` | If the object has no docstring |"""

print(actual)
assert actual == expected

Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'


| Type | Description |
| --- | --- |
| `ValueError` | If the object has no docstring |
| `HTTPError` | If the object has no docstring |


In [ ]:
# | export

def _format_return_section(
    section_dict: Dict[str, Union[str, List[DocstringRaise]]],
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    ret_val = """| Type | Description |
| --- | --- |
"""
    sig = signature(symbol)
    return_type = "`" + _get_return_annotation(sig, "").replace("->", "").replace("\n", "").strip() + "`"

    ret_val += "\n".join(
        [
            f"| {return_type} | {v.as_dict()['description']} |" # type: ignore
            for v in section_dict["value"]
        ]
    )
    
    return ret_val

In [ ]:
def fixture_function() -> str:
    """Test funcion
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Returns:
        A string
    """
    pass

docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[2].as_dict()

actual = _format_return_section(item_dict, fixture_function)
expected = """| Type | Description |
| --- | --- |
| `str` | A string |"""

print(actual)
assert actual == expected

Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:7: No type or annotation for returned value 1


| Type | Description |
| --- | --- |
| `str` | A string |


In [ ]:
def fixture_function() -> List[str]:
    """Test funcion
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Yields:
        A list of strings
    """
    pass

docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[2].as_dict()

actual = _format_return_section(item_dict, fixture_function)
expected = """| Type | Description |
| --- | --- |
| `List[str]` | A list of strings |"""

print(actual)
assert actual == expected


Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:7: No type or annotation for yielded value 1


| Type | Description |
| --- | --- |
| `List[str]` | A list of strings |


In [ ]:
# | export


def _get_default_value(default_value: List[str], param_name: str) -> str:
    if param_name == "**kwargs":
        return "`{}`"
    return "`required`" if len(default_value) == 0 else f"`{default_value[0].strip()}`"


def _format_params_annotations(
    params_annotation_list: List[str],
) -> Dict[str, Dict[str, str]]:
    ret_val = {}
    for param in params_annotation_list:
        if param == "self":
            continue
        param_name, annotation_with_default_value = param.split(":")
        annotation, *default_value = annotation_with_default_value.split("=")
        ret_val[param_name.strip()] = {
            "annotation": f"`{annotation.strip()}`",
            "default": _get_default_value(default_value, param_name),
        }
    return ret_val

In [ ]:
fixture = ['self', 'brand: str', 'model: str', 'type: str']
expected = {
    "brand": {
        "annotation": "`str`",
        "default": "`required`"
    },
    "model": {
        "annotation": "`str`",
        "default": "`required`"
    },
    "type": {
        "annotation": "`str`",
        "default": "`required`"
    }
}

actual = _format_params_annotations(fixture)
print(actual)
assert actual == expected


{'brand': {'annotation': '`str`', 'default': '`required`'}, 'model': {'annotation': '`str`', 'default': '`required`'}, 'type': {'annotation': '`str`', 'default': '`required`'}}


In [ ]:
fixture = ["o: Any", "supress_stdout: bool = False", "supress_stderr: bool = False", "sub_dict: Optional[Dict[str, str]] = None", "width: Optional[int] = 80", "**kwargs: str"]
expected = {
    "o": {
        "annotation": "`Any`",
        "default": "`required`"
    },
    "supress_stdout": {
        "annotation": "`bool`",
        "default": "`False`"
    },
    "supress_stderr": {
        "annotation": "`bool`",
        "default": "`False`"
    },
    "sub_dict": {
        "annotation": "`Optional[Dict[str, str]]`",
        "default": "`None`"
    },
    "width": {
        "annotation": "`Optional[int]`",
        "default": "`80`"
    },
    "**kwargs": {
        "annotation": "`str`",
        "default": "`{}`"
    }
}

actual = _format_params_annotations(fixture)
print(actual)
assert actual == expected


{'o': {'annotation': '`Any`', 'default': '`required`'}, 'supress_stdout': {'annotation': '`bool`', 'default': '`False`'}, 'supress_stderr': {'annotation': '`bool`', 'default': '`False`'}, 'sub_dict': {'annotation': '`Optional[Dict[str, str]]`', 'default': '`None`'}, 'width': {'annotation': '`Optional[int]`', 'default': '`80`'}, '**kwargs': {'annotation': '`str`', 'default': '`{}`'}}


In [ ]:
fixture = ["param_1: str", "param_2: int", "param_3: bool = True", "param_4: Optional[str] = None"]
expected = {
    "param_1": {
        "annotation": "`str`",
        "default": "`required`"
    },
    "param_2": {
        "annotation": "`int`",
        "default": "`required`"
    },
    "param_3": {
        "annotation": "`bool`",
        "default": "`True`"
    },
    "param_4": {
        "annotation": "`Optional[str]`",
        "default": "`None`"
    }
}

actual = _format_params_annotations(fixture)
print(actual)
assert actual == expected

{'param_1': {'annotation': '`str`', 'default': '`required`'}, 'param_2': {'annotation': '`int`', 'default': '`required`'}, 'param_3': {'annotation': '`bool`', 'default': '`True`'}, 'param_4': {'annotation': '`Optional[str]`', 'default': '`None`'}}


In [ ]:
# | export


def _format_param(
    param: DocstringParameter, param_annotations_dict: Dict[str, Dict[str, str]]
) -> str:
    param_dict = param.as_dict()
    nl = "\n"
    if param_dict['name'] in param_annotations_dict:
        ret_val =  f"| `{param_dict['name']}` | {param_annotations_dict[param_dict['name']]['annotation']} | {param_dict['description'].replace(nl, ' ')} | {param_annotations_dict[param_dict['name']]['default']} |" 
    else:
        ret_val =  f"| `{param_dict['name']}` |  | {param_dict['description'].replace(nl, ' ')} |  |"
        
    return ret_val

In [ ]:
def fixture_function(param_1: str, param_2: int, param_3: bool = True) -> List[str]:
    """Test funcion
    
    Args:
        param_1: string
        param_2: int
        param_3: boolean
        param_4: optional string
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Yields:
        A list of strings
    """
    pass

sig = signature(fixture_function)
param_annotations_str = _get_arg_list_with_signature(sig, True)
param_annotations_dict = _format_params_annotations(param_annotations_str)
    
docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[1].as_dict()
actual = []
expected = ["| `param_1` | `str` | string | `required` |",
"| `param_2` | `int` | int | `required` |",
"| `param_3` | `bool` | boolean | `True` |",
"| `param_4` |  | optional string |  |"]

for v in item_dict["value"]:
    actual.append(_format_param(v, param_annotations_dict))

print("\n".join(actual))
assert actual == expected

<module>:3: No type or annotation for parameter 'param_1'
<module>:4: No type or annotation for parameter 'param_2'
<module>:5: No type or annotation for parameter 'param_3'
<module>:6: No type or annotation for parameter 'param_4'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:13: No type or annotation for yielded value 1


| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` | `bool` | boolean | `True` |
| `param_4` |  | optional string |  |


In [ ]:
def fixture_function(param_1: str, param_2: int, param_3: bool = True, param_4: Optional[str] = None) -> List[str]:
    """Test funcion
    
    Args:
        param_1: string
        param_2: int
        param_3: boolean
        param_4: optional string
            new line
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Yields:
        A list of strings
    """
    pass

sig = signature(fixture_function)
param_annotations_str = _get_arg_list_with_signature(sig, True)
param_annotations_dict = _format_params_annotations(param_annotations_str)
    
docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[1].as_dict()
actual = []
expected = ["| `param_1` | `str` | string | `required` |",
"| `param_2` | `int` | int | `required` |",
"| `param_3` | `bool` | boolean | `True` |",
"| `param_4` | `Optional[str]` | optional string new line | `None` |"]

for v in item_dict["value"]:
    actual.append(_format_param(v, param_annotations_dict))

print("\n".join(actual))
assert actual == expected

<module>:3: No type or annotation for parameter 'param_1'
<module>:4: No type or annotation for parameter 'param_2'
<module>:5: No type or annotation for parameter 'param_3'
<module>:6: No type or annotation for parameter 'param_4'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:14: No type or annotation for yielded value 1


| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` | `bool` | boolean | `True` |
| `param_4` | `Optional[str]` | optional string new line | `None` |


In [ ]:
# | export


def _format_parameters_section(
    section_dict: Dict[str, Union[str, List[DocstringParameter]]],
    symbol: Union[types.FunctionType, Type[Any]],
) -> str:
    ret_val = """| Name | Type | Description | Default |
| --- | --- | --- | --- |
"""
    sig = signature(symbol)
    param_annotations_list = _get_arg_list_with_signature(sig, True)
    formatted_param_annotations_dict = _format_params_annotations(
        param_annotations_list # type: ignore
    )
    ret_val += "\n".join(
        [
            _format_param(v, formatted_param_annotations_dict) # type: ignore
            for v in section_dict["value"]
        ]
    )

    return ret_val

In [ ]:
def fixture_function(param_1: str, param_2: int, param_3: bool = True, param_4: Optional[str] = None) -> List[str]:
    """Test funcion
    
    Args:
        param_1: string
        param_2: int
        param_3: boolean
        param_4: optional string
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Yields:
        A list of strings
    """
    pass

docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[1].as_dict()

actual = _format_parameters_section(item_dict, fixture_function)
expected = """| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` | `bool` | boolean | `True` |
| `param_4` | `Optional[str]` | optional string | `None` |"""

print(actual)
assert actual == expected


<module>:3: No type or annotation for parameter 'param_1'
<module>:4: No type or annotation for parameter 'param_2'
<module>:5: No type or annotation for parameter 'param_3'
<module>:6: No type or annotation for parameter 'param_4'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:13: No type or annotation for yielded value 1


| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` | `bool` | boolean | `True` |
| `param_4` | `Optional[str]` | optional string | `None` |


In [ ]:
def fixture_function(param_1: str, param_2: int, param_3: bool = True, param_4: Optional[str] = None) -> List[str]:
    """Test funcion
    
    Args:
        param_1: string
        param_2: int
        param_3: boolean
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Yields:
        A list of strings
    """
    pass

docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[1].as_dict()

actual = _format_parameters_section(item_dict, fixture_function)
expected = """| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` | `bool` | boolean | `True` |"""

print(actual)
assert actual == expected


<module>:3: No type or annotation for parameter 'param_1'
<module>:4: No type or annotation for parameter 'param_2'
<module>:5: No type or annotation for parameter 'param_3'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:12: No type or annotation for yielded value 1


| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` | `bool` | boolean | `True` |


In [ ]:
def fixture_function(param_1: str, param_2: int) -> List[str]:
    """Test funcion
    
    Args:
        param_1: string
        param_2: int
        param_3: boolean
    
    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Yields:
        A list of strings
    """
    pass

docstring = Docstring(fixture_function.__doc__)
section_items = parse(docstring, Parser.google)
item_dict = section_items[1].as_dict()

actual = _format_parameters_section(item_dict, fixture_function)
expected = """| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` |  | boolean |  |"""

print(actual)
assert actual == expected


<module>:3: No type or annotation for parameter 'param_1'
<module>:4: No type or annotation for parameter 'param_2'
<module>:5: No type or annotation for parameter 'param_3'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:12: No type or annotation for yielded value 1


| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `param_1` | `str` | string | `required` |
| `param_2` | `int` | int | `required` |
| `param_3` |  | boolean |  |


In [ ]:
# | export

def _get_source_relative_filename(source_filename: Optional[str] = None) -> Optional[str]:
    if source_filename is None:
        return None
    cfg = get_config()
    lib_name = cfg["lib_name"].replace("-", "_")
    return f"{lib_name}{source_filename.split(lib_name)[1]}"

In [ ]:
source_filename = getsourcefile(prepare)
actual = _get_source_relative_filename(source_filename)
expected = "nbdev_mkdocs/mkdocs.py"
print(actual)
assert actual == expected

nbdev_mkdocs/mkdocs.py


In [ ]:
# | export

def _get_source_code(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    return getsource(symbol)

In [ ]:
def fixture_function(i: str, a: str) -> str:
    """This is a test function
    """
    pass

expected = 'def fixture_function(i: str, a: str) -> str:\n    """This is a test function\n    """\n    pass\n'
actual = _get_source_code(fixture_function)
display(actual)
assert actual == expected

'def fixture_function(i: str, a: str) -> str:\n    """This is a test function\n    """\n    pass\n'

In [ ]:
# | export


def _docstring_to_markdown(symbol: Union[types.FunctionType, Type[Any]]) -> str:
    """Converts a docstring to a markdown-formatted string.

    Args:
        docstring: The docstring to convert.

    Returns:
        The markdown-formatted docstring.
    """
    docstring = Docstring(symbol.__doc__)  # type: ignore
    parsed_docstring_sections = parse(docstring, Parser.google)
    formatted_docstring = ""
    for section in parsed_docstring_sections:
        section_dict = section.as_dict()
        if section_dict["kind"] == "text":
            formatted_docstring += f"{section.value}\n\n"

        elif section_dict["kind"] == "examples":
            section_header = f"**{section_dict['kind'].capitalize()}**:\n\n"
            formatted_docstring += f"{section_header}" + "".join(
                [f"{v[1]}\n\n" for v in section_dict["value"]]
            )

        elif section_dict["kind"] == "returns" or section_dict["kind"] == "yields":
            section_header = f"**{section_dict['kind'].capitalize()}**:\n\n"
            formatted_docstring += (
                f"{section_header}"
                + _format_return_section(section_dict, symbol)
                + "\n\n"
            )

        elif section_dict["kind"] == "raises":
            section_header = f"**{section_dict['kind'].capitalize()}**:\n\n"
            formatted_docstring += (
                f"{section_header}" + _format_raises_section(section_dict) + "\n\n"
            )
        elif section_dict["kind"] == "parameters":
            section_header = f"**{section_dict['kind'].capitalize()}**:\n\n"
            formatted_docstring += (
                f"{section_header}"
                + _format_parameters_section(section_dict, symbol)
                + "\n\n"
            )
        elif section_dict["kind"] == "admonition":
            value = section_dict["value"]
            annotation = value["annotation"]
            description = value["description"]
            formatted_docstring += (
                f"??? {annotation}"
                + "\n\n"
                + f"{textwrap.indent(description, '    ')}\n\n"
            )

    source_relative_filename = _get_source_relative_filename(getsourcefile(symbol))
    if source_relative_filename is not None:
        source_code = _get_source_code(symbol)
        formatted_docstring += (
            f'??? quote "Source code in `{source_relative_filename}`"\n\n'
            + f"    ```python\n{textwrap.indent(source_code, '    ')}    ```\n\n"
        )

    return formatted_docstring

In [ ]:
# from inspect import getfile

getsourcefile_mock_value = '/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py'

@contextmanager
def mock_getsourcefile():
    with unittest.mock.patch('__main__.getsourcefile') as mock_getsourcefile:
        mock_getsourcefile.return_value = getsourcefile_mock_value
        yield
        

def fixture_function(i: str, a: str) -> str:
    pass

# print(getfile(fixture_function))

with mock_getsourcefile():
    actual = getsourcefile(fixture_function)
    print(actual)
    assert actual == getsourcefile_mock_value
    
class Fixture:
    pass

with mock_getsourcefile():
    actual = getsourcefile(Fixture)
    print(actual)
    assert actual == getsourcefile_mock_value

/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py
/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py


In [ ]:
getsource_mock_value = 'class Test:\n    pass'
mock_getsourcefile_value = '/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py'

@contextmanager
def mock_getsource():
    with unittest.mock.patch('__main__.getsource') as mock_getsource:
#         with unittest.mock.patch('__main__.getsourcefile') as mock_getsourcefile:
        mock_getsource.return_value = getsource_mock_value
#             mock_getsourcefile.return_value = mock_getsourcefile_value
        yield
        

class Fixture:
    pass

# with mock_getsourcefile():
#     actual = getsourcefile(Fixture)
#     print(actual)
#     assert actual == "/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py"

    
with mock_getsource():
    actual = getsource(Fixture)
    print(actual)
    assert actual == getsource_mock_value
    
    with mock_getsourcefile():
        actual = getsourcefile(Fixture)
        print(actual)
        assert actual == getsourcefile_mock_value


class Test:
    pass
/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py


In [ ]:
class Fixture:
    pass

with mock_getsourcefile():
    actual = getsourcefile(Fixture)
    print(actual)
    assert actual == getsourcefile_mock_value

/Users/harishm/Dev/airt-git/nbdev-mkdocs/nbdev_mkdocs/mkdocs.py


In [ ]:
def fixture_function(i: str, a: str) -> str:
    """Very cool function

    **f** is a very cool function
    
    Note:
        Execution context is not the same as the one in the notebook because we want examples to work from
        user code. Make sure you compiled the library prior to executing the examples, otherwise you might
        be running them agains an old version of the library.

    Args:
        i: something
        a: something else
        
    Returns:
        This function returns a sample string

    Raises:
        ValueError: If the object has no docstring
        HTTPError: If the object has no docstring
        
    Examples:
        The following snippet prints out greetings for two names:
        ```python
        print("hello {fill in name_1}")
        print("goodbye {fill in name_2}")
        ```
        
    Example:
        The following snippet prints out greetings for two names:
        ```python
        print("hello {fill in name_1}")
        print("goodbye {fill in name_2}")
        ```
        
    Yields:
        This functiron yields something

    !!! note

        The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)

    """
    pass

expected = """Very cool function

**f** is a very cool function

??? note

    Execution context is not the same as the one in the notebook because we want examples to work from
    user code. Make sure you compiled the library prior to executing the examples, otherwise you might
    be running them agains an old version of the library.

**Parameters**:

| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `i` | `str` | something | `required` |
| `a` | `str` | something else | `required` |

**Returns**:

| Type | Description |
| --- | --- |
| `str` | This function returns a sample string |

**Raises**:

| Type | Description |
| --- | --- |
| `ValueError` | If the object has no docstring |
| `HTTPError` | If the object has no docstring |

**Examples**:

The following snippet prints out greetings for two names:
```python
print("hello {fill in name_1}")
print("goodbye {fill in name_2}")
```

??? example

    The following snippet prints out greetings for two names:
    ```python
    print("hello {fill in name_1}")
    print("goodbye {fill in name_2}")
    ```

**Yields**:

| Type | Description |
| --- | --- |
| `str` | This functiron yields something |

!!! note

    The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    def fixture_function(i: str, a: str) -> str:
        \"\"\"Very cool function

        **f** is a very cool function
    
        Note:
            Execution context is not the same as the one in the notebook because we want examples to work from
            user code. Make sure you compiled the library prior to executing the examples, otherwise you might
            be running them agains an old version of the library.

        Args:
            i: something
            a: something else
        
        Returns:
            This function returns a sample string

        Raises:
            ValueError: If the object has no docstring
            HTTPError: If the object has no docstring
        
        Examples:
            The following snippet prints out greetings for two names:
            ```python
            print("hello {fill in name_1}")
            print("goodbye {fill in name_2}")
            ```
        
        Example:
            The following snippet prints out greetings for two names:
            ```python
            print("hello {fill in name_1}")
            print("goodbye {fill in name_2}")
            ```
        
        Yields:
            This functiron yields something

        !!! note

            The above docstring is autogenerated by docstring-gen library (https://github.com/airtai/docstring-gen)

        \"\"\"
        pass
    ```

"""

with mock_getsourcefile():
    actual = _docstring_to_markdown(fixture_function)
    display(actual)

assert actual == expected

<module>:10: No type or annotation for parameter 'i'
<module>:11: No type or annotation for parameter 'a'
<module>:14: No type or annotation for returned value 1
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
Failed to parse annotation from 'Name' node: 'NoneType' object has no attribute 'resolve'
<module>:35: No type or annotation for yielded value 1


'Very cool function\n\n**f** is a very cool function\n\n??? note\n\n    Execution context is not the same as the one in the notebook because we want examples to work from\n    user code. Make sure you compiled the library prior to executing the examples, otherwise you might\n    be running them agains an old version of the library.\n\n**Parameters**:\n\n| Name | Type | Description | Default |\n| --- | --- | --- | --- |\n| `i` | `str` | something | `required` |\n| `a` | `str` | something else | `required` |\n\n**Returns**:\n\n| Type | Description |\n| --- | --- |\n| `str` | This function returns a sample string |\n\n**Raises**:\n\n| Type | Description |\n| --- | --- |\n| `ValueError` | If the object has no docstring |\n| `HTTPError` | If the object has no docstring |\n\n**Examples**:\n\nThe following snippet prints out greetings for two names:\n```python\nprint("hello {fill in name_1}")\nprint("goodbye {fill in name_2}")\n```\n\n??? example\n\n    The following snippet prints out greeti

In [ ]:
# | export


def get_formatted_docstring_for_symbol(
    symbol: Union[types.FunctionType, Type[Any]]
) -> str:
    """Recursively parses and get formatted docstring of a symbol.

    Args:
        symbol: A Python class or function object to parse the docstring for.

    Returns:
        A formatted docstring of the symbol and its members.

    """

    def traverse(symbol: Union[types.FunctionType, Type[Any]], contents: str) -> str:
        """Recursively traverse the members of a symbol and append their docstrings to the provided contents string.

        Args:
            symbol: A Python class or function object to parse the docstring for.
            contents: The current formatted docstrings.

        Returns:
            The updated formatted docstrings.

        """
        for x, y in getmembers(symbol):
            if not x.startswith("_") or x == "__init__":
                if isfunction(y) and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n\n{_docstring_to_markdown(y)}"
                elif isclass(y) and not x.startswith("__") and y.__doc__ is not None:
                    contents += f"{_get_symbol_definition(y)}\n\n{_docstring_to_markdown(y)}"
                    contents = traverse(y, contents)
        return contents

    contents = (
        f"{_get_symbol_definition(symbol)}\n\n{_docstring_to_markdown(symbol)}"
        if symbol.__doc__ is not None
        else ""
    )
    if isclass(symbol):
        contents = traverse(symbol, contents)
    return contents

In [ ]:
def fixture_function(
    arg_1: str,
    arg_2: Union[List[str], str],
    arg_3: Optional[int],
    arg_4: Optional[str] = None,
) -> str:
    """This is a one line description for the function

    Args:
        arg_1: Argument 1
        arg_2: Argument 2
        arg_3: Argument 3
        arg_4: Argument 4

    Returns:
        The concatinated string
    """
    pass


expected = """### `fixture_function`

`def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str
`

This is a one line description for the function

**Parameters**:

| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `arg_1` | `str` | Argument 1 | `required` |
| `arg_2` | `Union[List[str], str]` | Argument 2 | `required` |
| `arg_3` | `Optional[int]` | Argument 3 | `required` |
| `arg_4` | `Optional[str]` | Argument 4 | `None` |

**Returns**:

| Type | Description |
| --- | --- |
| `str` | The concatinated string |

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    def fixture_function(
        arg_1: str,
        arg_2: Union[List[str], str],
        arg_3: Optional[int],
        arg_4: Optional[str] = None,
    ) -> str:
        \"\"\"This is a one line description for the function

        Args:
            arg_1: Argument 1
            arg_2: Argument 2
            arg_3: Argument 3
            arg_4: Argument 4

        Returns:
            The concatinated string
        \"\"\"
        pass
    ```

"""

with mock_getsourcefile():
    actual = get_formatted_docstring_for_symbol(fixture_function)
    display(actual)

assert actual == expected

<module>:3: No type or annotation for parameter 'arg_1'
<module>:4: No type or annotation for parameter 'arg_2'
<module>:5: No type or annotation for parameter 'arg_3'
<module>:6: No type or annotation for parameter 'arg_4'
<module>:9: No type or annotation for returned value 1


'### `fixture_function`\n\n`def fixture_function(arg_1: str, arg_2: Union[List[str], str], arg_3: Optional[int], arg_4: Optional[str] = None) -> str\n`\n\nThis is a one line description for the function\n\n**Parameters**:\n\n| Name | Type | Description | Default |\n| --- | --- | --- | --- |\n| `arg_1` | `str` | Argument 1 | `required` |\n| `arg_2` | `Union[List[str], str]` | Argument 2 | `required` |\n| `arg_3` | `Optional[int]` | Argument 3 | `required` |\n| `arg_4` | `Optional[str]` | Argument 4 | `None` |\n\n**Returns**:\n\n| Type | Description |\n| --- | --- |\n| `str` | The concatinated string |\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    def fixture_function(\n        arg_1: str,\n        arg_2: Union[List[str], str],\n        arg_3: Optional[int],\n        arg_4: Optional[str] = None,\n    ) -> str:\n        """This is a one line description for the function\n\n        Args:\n            arg_1: Argument 1\n            arg_2: Argument 2\n         

In [ ]:
class Vehicle:
    """This is a docstring for the class"""

    def __init__(self, brand: str, model: str, type: str):
        """Constructor

        Args:
            brand: Name of the brand
            model: Name of the model
            type: Model type
        """
        self.brand = brand
        self.model = model
        self.type = type
        self.gas_tank_size = 14
        self.fuel_level = 0
        self.speed = 1

    def fuel_up(self):
        """Fuel up"""
        self.fuel_level = self.gas_tank_size
        print("Gas tank is now full.")

    def drive(self, speed: int = 10):
        """Drive
        
        Args:
            speed: drive speed
        """
        self.speed = speed
        print(f"The {self.model} is now driving at {self.speed}.")


expected = """

This is a docstring for the class

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

### `__init__`

`def __init__(self, brand: str, model: str, type: str) -> None
`

Constructor

**Parameters**:

| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `brand` | `str` | Name of the brand | `required` |
| `model` | `str` | Name of the model | `required` |
| `type` | `str` | Model type | `required` |

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

### `drive`

`def drive(self, speed: int = 10) -> None
`

Drive

**Parameters**:

| Name | Type | Description | Default |
| --- | --- | --- | --- |
| `speed` | `int` | drive speed | `10` |

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

### `fuel_up`

`def fuel_up(self) -> None
`

Fuel up

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

"""


with mock_getsourcefile():
    with mock_getsource():
        actual = get_formatted_docstring_for_symbol(Vehicle)

display(actual)
assert actual == expected

<module>:3: No type or annotation for parameter 'brand'
<module>:4: No type or annotation for parameter 'model'
<module>:5: No type or annotation for parameter 'type'
<module>:3: No type or annotation for parameter 'speed'


'\n\nThis is a docstring for the class\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n### `__init__`\n\n`def __init__(self, brand: str, model: str, type: str) -> None\n`\n\nConstructor\n\n**Parameters**:\n\n| Name | Type | Description | Default |\n| --- | --- | --- | --- |\n| `brand` | `str` | Name of the brand | `required` |\n| `model` | `str` | Name of the model | `required` |\n| `type` | `str` | Model type | `required` |\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n### `drive`\n\n`def drive(self, speed: int = 10) -> None\n`\n\nDrive\n\n**Parameters**:\n\n| Name | Type | Description | Default |\n| --- | --- | --- | --- |\n| `speed` | `int` | drive speed | `10` |\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n### `fuel_up`\n\n`def fuel_up(self) -> None\n`\n\nFuel up\n\n??? quote "Source c

In [ ]:
class Outer:
    """Outer Class"""

    def __init__(self):
        """Outer class constructor"""
        ## instantiating the 'Inner' class
        self.inner = self.Inner()

    def reveal(self):
        """Reveal function"""
        ## calling the 'Inner' class function display
        self.inner.inner_display("Calling Inner class function from Outer class")

    class Inner:
        """Inner Class"""

        def inner_display(self, msg):
            """Inner display"""
            print(msg)

    class Inner2:
        """Inner2 Class"""

        def inner_display_2(self, msg):
            """Inner display_2"""
            print(msg)


expected = """

Outer Class

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```



Inner Class

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

### `inner_display`

`def inner_display(self, msg) -> None
`

Inner display

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```



Inner2 Class

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

### `inner_display_2`

`def inner_display_2(self, msg) -> None
`

Inner display_2

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

### `__init__`

`def __init__(self) -> None
`

Outer class constructor

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

### `reveal`

`def reveal(self) -> None
`

Reveal function

??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"

    ```python
    class Test:
        pass    ```

"""

with mock_getsourcefile():
    with mock_getsource():
        actual = get_formatted_docstring_for_symbol(Outer)

display(actual)
assert actual == expected

'\n\nOuter Class\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n\n\nInner Class\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n### `inner_display`\n\n`def inner_display(self, msg) -> None\n`\n\nInner display\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n\n\nInner2 Class\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n### `inner_display_2`\n\n`def inner_display_2(self, msg) -> None\n`\n\nInner display_2\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n### `__init__`\n\n`def __init__(self) -> None\n`\n\nOuter class constructor\n\n??? quote "Source code in `nbdev_mkdocs/mkdocs.py`"\n\n    ```python\n    class Test:\n        pass    ```\n\n### `reveal`\n\n`def reveal(